In [51]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [52]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [53]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [54]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [55]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [56]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [57]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

In [58]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [59]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [60]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

col = ['요일']
target=['연령대','시간대']
def drop_filter(df1,target):
    df_a=df1.drop(df1.filter(regex='|'.join(target)),axis=1)
    return df_a 

def multi_merge(a_df, b_df):
    all_df = pd.merge(left=a_df, right=b_df, how='left', on=['기준_년_분기_코드','상권_코드'], sort=False)
    all_df = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df.dropna(axis=0)

In [61]:
# multi_columns_merging('상권-추정유동인구', '상권배후지-추정유동인구')

In [62]:
f_pop=multi_columns_merging('상권-추정유동인구', '상권배후지-추정유동인구')
# f_pop.info()

In [63]:
target1=['hl_월','hl_화','hl_수','hl_목','hl_금']
target2=['hl_토','hl_일']
def add_week(df,target1,target2):
    df['hl_주중_유동인구_수']=df.filter(regex='|'.join(target1)).sum(axis=1)
    df['hl_주말_유동인구_수']=df.filter(regex='|'.join(target2)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target1+target2)),axis=1)
    return df_a

target3=['월요','화요','수요','목요','금요']
target4=['토요','일요']
def add_week2(df,target3,target4):
    df['주중_유동인구_수']=df.filter(regex='|'.join(target3)).sum(axis=1)
    df['주말_유동인구_수']=df.filter(regex='|'.join(target4)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target3+target4)),axis=1)
    return df_a

# hl을 먼저 합해라!

In [14]:
f_pop_a=add_week(f_pop,target1,target2)
# f_pop_a

In [15]:
flow_pop=add_week2(f_pop_a,target3,target4)

In [16]:
# flow_pop.columns

In [17]:
# 컬럼 정렬
flow_pop=flow_pop[['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수','주중_유동인구_수', '주말_유동인구_수',
       'hl_총_유동인구_수', 'hl_남성_유동인구_수', 'hl_여성_유동인구_수', 'hl_주중_유동인구_수','hl_주말_유동인구_수']]

In [18]:
# flow_pop.head()

In [19]:
# 비율 구하기
def ratio(df,col1,col2):
    total = df[col1]+df[col2]
    total2 = df[col1]/total*100
    return total2

In [20]:
flow_pop['남성_유동인구_비율']=ratio(flow_pop,'남성_유동인구_수', '여성_유동인구_수')
flow_pop['여성_유동인구_비율']=ratio(flow_pop,'여성_유동인구_수', '남성_유동인구_수')
flow_pop['주중_유동인구_비율']=ratio(flow_pop,'주중_유동인구_수', '주말_유동인구_수')
flow_pop['주말_유동인구_비율']=ratio(flow_pop,'주말_유동인구_수', '주중_유동인구_수')
flow_pop['hl_남성_유동인구_비율']=ratio(flow_pop,'hl_남성_유동인구_수', 'hl_여성_유동인구_수')
flow_pop['hl_여성_유동인구_비율']=ratio(flow_pop,'hl_여성_유동인구_수', 'hl_남성_유동인구_수')
flow_pop['hl_주중_유동인구_비율']=ratio(flow_pop,'hl_주중_유동인구_수', 'hl_주말_유동인구_수')
flow_pop['hl_주말_유동인구_비율']=ratio(flow_pop,'hl_주말_유동인구_수', 'hl_주중_유동인구_수')

In [40]:
flow_pop.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,주중_유동인구_수,주말_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,hl_주중_유동인구_수,hl_주말_유동인구_수,남성_유동인구_비율,여성_유동인구_비율,주중_유동인구_비율,주말_유동인구_비율,hl_남성_유동인구_비율,hl_여성_유동인구_비율,hl_주중_유동인구_비율,hl_주말_유동인구_비율
0,2019_1,1000001,118266,61268,56997,86815,31451,471903.0,253713.0,218188.0,359301.0,112607.0,51.805691,48.194309,73.406558,26.593442,53.764031,46.235969,76.137934,23.862066
1,2019_1,1000002,246619,147358,99261,156385,90234,2186033.0,1237827.0,948202.0,1499105.0,686921.0,59.751276,40.248724,63.411578,36.588422,56.624455,43.375545,68.576723,31.423277
2,2019_1,1000003,144480,80073,64407,107665,36815,1818586.0,1015350.0,803234.0,1340682.0,477893.0,55.421512,44.578488,74.518965,25.481035,55.831900,44.168100,73.721568,26.278432
3,2019_1,1000004,322387,155635,166753,248361,74025,1208957.0,573717.0,635243.0,914237.0,294728.0,48.275680,51.724320,77.038395,22.961605,47.455416,52.544584,75.621461,24.378539
4,2019_1,1000005,23622,10781,12848,14580,9041,47589.0,22232.0,25363.0,30616.0,16966.0,45.626137,54.373863,61.724736,38.275264,46.710789,53.289211,64.343659,35.656341


In [22]:
# pop_a=pop.drop(pop.filter(regex='연령대|시간대'),axis=1)
# hl_pop_a=hl_pop.drop(hl_pop.filter(regex='연령대|시간대'),axis=1)

In [23]:
stay_pop=multi_columns_merging('상권-상주인구', '상권배후지-상주인구')
stay_pop.head()

,기준_년_분기_코드,상권_코드,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,hl_비_아파트_가구_수
471,2019_2,1001010,3088,1526,1562,1601,0,1601,28320.0,13999.0,14321.0,13046.0,1888.0,11158.0
472,2019_2,1001009,4032,2056,1976,2067,39,2028,22242.0,10954.0,11288.0,10288.0,1770.0,8518.0
473,2019_2,1001008,2323,1114,1209,1261,0,1261,16300.0,8141.0,8159.0,8379.0,265.0,8114.0
474,2019_2,1001007,2208,1105,1103,980,0,980,5262.0,2676.0,2586.0,2175.0,182.0,1993.0
475,2019_2,1001006,4523,2227,2296,1738,531,1207,21885.0,10817.0,11068.0,8598.0,3081.0,5517.0


In [24]:
office_pop=multi_columns_merging('상권-직장인구','상권배후지-직장인구')
office_pop.head()

,기준_년_분기_코드,상권_코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,hl_총_직장_인구_수,hl_남성_직장_인구_수,hl_여성_직장_인구_수
483,2019_2,1001010,231,73,158,2935.0,1774.0,1161.0
484,2019_2,1001009,543,378,165,6413.0,3834.0,2579.0
485,2019_2,1001008,149,95,54,4362.0,2489.0,1873.0
486,2019_2,1001007,462,305,157,1171.0,816.0,355.0
487,2019_2,1001006,504,261,243,1996.0,1108.0,888.0


In [25]:
# delete_word_columns_1 = f_pop_a.filter(like='대_').columns
# f_pop_a.drop(delete_word_columns_1, axis=1, inplace=True)

In [26]:
ch_df=load_to_df('상권-상권변화지표')
ch_19_df=ch_df[ch_df['기준_년_분기_코드'].str.contains('2019_1|2019_2')]
ch_19_name=ch_19_df[['기준_년_분기_코드','상권_코드','상권_변화_지표']]

In [27]:
# ch_19_df[['기준_년_분기_코드','상권_코드','상권_변화_지표']].head()

In [28]:
# flow_pop.shape , stay_pop.shape, office_pop.shape

In [29]:
people_a = multi_merge(flow_pop,stay_pop)
people = multi_merge(people_a,office_pop)   #유동,상주,직장인구 총 데이터 merge 

In [30]:
f_pop_ch=multi_merge(flow_pop, ch_19_name)   #유동인구 + 지표
s_pop_ch=multi_merge(stay_pop, ch_19_name)   #상주인구 + 지표
o_pop_ch=multi_merge(office_pop, ch_19_name) #직장인구 + 지표
total_people=multi_merge(people, ch_19_name)  #총 인구 + 지표

In [31]:
f_pop_ch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 0 to 2019
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   기준_년_분기_코드     2020 non-null   object 
 1   상권_코드          2020 non-null   object 
 2   총_유동인구_수       2020 non-null   int64  
 3   남성_유동인구_수      2020 non-null   int64  
 4   여성_유동인구_수      2020 non-null   int64  
 5   주중_유동인구_수      2020 non-null   int64  
 6   주말_유동인구_수      2020 non-null   int64  
 7   hl_총_유동인구_수    2020 non-null   float64
 8   hl_남성_유동인구_수   2020 non-null   float64
 9   hl_여성_유동인구_수   2020 non-null   float64
 10  hl_주중_유동인구_수   2020 non-null   float64
 11  hl_주말_유동인구_수   2020 non-null   float64
 12  남성_유동인구_비율     2020 non-null   float64
 13  여성_유동인구_비율     2020 non-null   float64
 14  주중_유동인구_비율     2020 non-null   float64
 15  주말_유동인구_비율     2020 non-null   float64
 16  hl_남성_유동인구_비율  2020 non-null   float64
 17  hl_여성_유동인구_비율  2020 non-null   float64
 18  hl_주중_유동

In [32]:
# pd.get_dummies(f_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

In [33]:
# pd.get_dummies(s_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

In [34]:
# pd.get_dummies(o_pop_ch, columns=['상권_변화_지표'], drop_first=False).head()

In [35]:
# pd.get_dummies(total_people, columns=['상권_변화_지표'], drop_first=False)

In [49]:
# flow_pop_ch=pd.get_dummies(f_pop_ch, columns=['상권_변화_지표'], drop_first=False)
# flow_pop_ch

In [48]:
# ch_name_19=pd.get_dummies(ch_19_name, columns=['상권_변화_지표'], drop_first=False)
# ch_name_19

In [38]:
from sklearn.model_selection import train_test_split

In [44]:
# train = ch_name_19.copy()
# test = flow_pop_ch.copy()
# flow_pop_1 = flow_pop[flow_pop['기준_년_분기_코드']=='2019_1']
# flow_pop_2 = flow_pop[flow_pop['기준_년_분기_코드']=='2019_2']

In [50]:
f_pop_ch.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,주중_유동인구_수,주말_유동인구_수,hl_총_유동인구_수,hl_남성_유동인구_수,hl_여성_유동인구_수,...,hl_주말_유동인구_수,남성_유동인구_비율,여성_유동인구_비율,주중_유동인구_비율,주말_유동인구_비율,hl_남성_유동인구_비율,hl_여성_유동인구_비율,hl_주중_유동인구_비율,hl_주말_유동인구_비율,상권_변화_지표
0,2019_1,1000001,118266,61268,56997,86815,31451,471903.0,253713.0,218188.0,...,112607.0,51.805691,48.194309,73.406558,26.593442,53.764031,46.235969,76.137934,23.862066,LH
1,2019_1,1000002,246619,147358,99261,156385,90234,2186033.0,1237827.0,948202.0,...,686921.0,59.751276,40.248724,63.411578,36.588422,56.624455,43.375545,68.576723,31.423277,HL
2,2019_1,1000003,144480,80073,64407,107665,36815,1818586.0,1015350.0,803234.0,...,477893.0,55.421512,44.578488,74.518965,25.481035,55.831900,44.168100,73.721568,26.278432,HH
3,2019_1,1000004,322387,155635,166753,248361,74025,1208957.0,573717.0,635243.0,...,294728.0,48.275680,51.724320,77.038395,22.961605,47.455416,52.544584,75.621461,24.378539,LH
4,2019_1,1000005,23622,10781,12848,14580,9041,47589.0,22232.0,25363.0,...,16966.0,45.626137,54.373863,61.724736,38.275264,46.710789,53.289211,64.343659,35.656341,LL


((1010, 20), (1010, 20))